In [16]:
#!pip install psycopg2

In [17]:
import psycopg2
connection = psycopg2.connect(
    dbname='testdb',
    user='postgres',
    password='1234',
    host='193.203.191.79',
    port='32001'
)
query = "Select * from users"

In [18]:
import pandas as pd

df = pd.read_sql(query, connection)
connection.close()

C:\Users\PC1\AppData\Local\Temp\ipykernel_54156\3020219344.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, connection)


In [19]:
df.head(1)

,id,name,email,review,rating
0,1,Raymond Howard,erodriguez@example.net,Partner trouble serious sign bar follow. Assum...,6.8


In [20]:
texts = df['review'].values
ratings = df['rating'].values

In [21]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(texts,ratings,test_size=0.3,random_state=42)

In [23]:
#!pip install tensorflow

In [26]:
from tensorflow.keras.preprocessing.text import Tokenizer # alternatifler?

tokenizer = Tokenizer(num_words=10000, oov_token='<OOV>') # Out-of Vocabulary
tokenizer.fit_on_texts(X_train)

In [28]:
#tokenizer.word_index
# last idea -> [ 383, 382 ]

In [33]:
X_train_seq = tokenizer.texts_to_sequences(X_train)
X_test_seq = tokenizer.texts_to_sequences(X_test)

In [46]:
# Padding ? => Bütün cümleleri aynı boyuta getirme.
# Bir boyut belirle. Her cümleyi bu boyuta getir. Uzunsa kırpıyoruz => gerekirse 2-3 arraye çevir [50] [50] [50]
# 39 0 0 0 0 0x11
from tensorflow.keras.preprocessing.sequence import pad_sequences

X_train_pad = pad_sequences(X_train_seq, maxlen=50, padding="post") # post-> boşluklar sona | pre->boşluklar başa
X_test_pad = pad_sequences(X_test_seq, maxlen=50, padding="post")

In [49]:
# Modeli kur?
import tensorflow as tf

model = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim=10000, output_dim=64, input_length=50), # kelimeleri vektörlere çevir.
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)), # LSTM (64 nöronluk) Long Short Term Memory # 64+64=>128
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dropout(0.4), # %40 oranında nöronları pasif halde çalıştırmak
    tf.keras.layers.Dense(1, activation="linear") # 1=> Çıktı adet sayısı (review skoru) #activation="linear" f(x) = x
])
model.compile(loss='mean_squared_error', optimizer='adam', metrics=['mae'])

# Activation fonksiyonu nedir? Farklı activation fonksiyonlarını araştıralım?
# Backpropagation (Geri Yayılım) nedir? Araştıralım.
# İlgili modeli öğrendiğimiz yeni bilgilerle geliştirelim. 

In [50]:
# Eğit
history = model.fit(X_train_pad, y_train, epochs=1, validation_data=(X_test_pad,y_test), batch_size=2)

1404/1404 ━━━━━━━━━━━━━━━━━━━━ 12s 8ms/step - loss: 11.2466 - mae: 2.7678 - val_loss: 6.9641 - val_mae: 2.2585


In [35]:
# val_loss => Doğrulama verisine göre
# loss => Eğitim verisine göre.

In [51]:
model.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_1 (Embedding)              │ (None, 50, 64)              │         640,000 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ bidirectional_1 (Bidirectional)      │ (None, 128)                 │          66,048 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 32)                  │           4,128 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 32)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 1)                   │              33 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 2,130,629 (8.13 MB)

 Trainable params: 710,209 (2.71 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 1,420,420 (5.42 MB)